In [1]:
# AI Sales Agent - Google Colab Notebook

# ✅ **Step 1: Install Required Libraries**
!pip install predictionguard langchain langchain_community pandas python-dotenv cryptography


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [11]:
# ✅ **Step 2: Import Required Libraries**
import os
import json
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
from cryptography.fernet import Fernet
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import PredictionGuard

In [4]:
# ✅ **Step 3: Secure API Key Input**
load_dotenv()


False

In [12]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

Enter your Prediction Guard access token: ··········


In [14]:
# ✅ **Step 4: Initialize Prediction Guard Model**
llm = PredictionGuard(
    model="Hermes-3-Llama-3.1-70B",
    predictionguard_api_key=os.environ['PREDICTIONGUARD_TOKEN'],
    temperature=0.75,
    max_tokens=100,
    stop=["000"]
)

In [15]:
# ✅ **Step 5: Meeting Notes (Example Input for Processing)**
meeting_notes = [
    {
        "date": "2025-02-05",
        "attendees": ["University IT Representative", "Prediction Guard Team"],
        "discussion": "The university is exploring self-hosted AI solutions to maintain data privacy..."
    },
    {
        "date": "2025-02-06",
        "attendees": ["CTO from Manufacturing Software Provider", "Prediction Guard Team"],
        "discussion": "The company wants a vertical AI solution for aerospace and defense MES..."
    }
]

In [16]:
# ✅ **Step 6: Encrypt and Decrypt Meeting Notes**
def generate_encryption_key():
    key = Fernet.generate_key()
    return key

def encrypt_meeting_notes(notes, key):
    cipher = Fernet(key)
    encrypted_data = cipher.encrypt(json.dumps(notes).encode("utf-8"))
    return encrypted_data

def decrypt_meeting_notes(encrypted_data, key):
    cipher = Fernet(key)
    decrypted_data = cipher.decrypt(encrypted_data).decode("utf-8")
    return json.loads(decrypted_data)

# Generate Encryption Key
enc_key = generate_encryption_key()

# Encrypt Notes
encrypted_notes = encrypt_meeting_notes(meeting_notes, enc_key)

# Decrypt Notes for Processing
decrypted_notes = decrypt_meeting_notes(encrypted_notes, enc_key)
print("✅ Meeting Notes Successfully Decrypted!")

✅ Meeting Notes Successfully Decrypted!


In [17]:
# ✅ **Step 7: Summarization and Next Steps Generation**
summary_prompt_template = PromptTemplate(
    input_variables=["notes"],
    template="Summarize the following meeting notes in bullet points:\n\n{notes}"
)

summarization_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def summarize_notes(notes):
    try:
        if not isinstance(notes, str) or notes.strip() == "":
            return "No valid notes available."
        summary = summarization_chain.run({"notes": notes})
        return summary.strip() if summary else "Summarization failed."
    except Exception as e:
        return f"Error: {str(e)}"

def deduce_next_steps(summary):
    if "self-hosted" in summary.lower():
        return "Follow up with security and IT teams for deployment feasibility."
    elif "budget concerns" in summary.lower():
        return "Schedule a cost-benefit discussion with financial stakeholders."
    elif "pilot agreement" in summary.lower():
        return "Send pilot agreement document and schedule next review meeting."
    else:
        return "Send follow-up email summarizing key discussion points."

<ipython-input-17-901560843971>:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarization_chain = LLMChain(llm=llm, prompt=summary_prompt_template)


In [18]:
# ✅ **Step 8: Process Meeting Notes**
processed_data = []

for meeting in decrypted_notes:
    summary = summarize_notes(meeting["discussion"])
    next_steps = deduce_next_steps(summary)

    processed_data.append({
        "Date": meeting["date"],
        "Attendees": ", ".join(meeting["attendees"]),
        "Summary": summary,
        "Next Steps": next_steps
    })

<ipython-input-17-901560843971>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summarization_chain.run({"notes": notes})


In [19]:
# ✅ **Step 9: Convert to DataFrame and Save**
df = pd.DataFrame(processed_data)
df.to_csv("sales_pipeline_summaries.csv", index=False)

print("✅ AI Sales Agent Processing Complete! Here are the results:")
df.head()

✅ AI Sales Agent Processing Complete! Here are the results:


,Date,Attendees,Summary,Next Steps
0,2025-02-05,"University IT Representative, Prediction Guard...",- The IT department is researching potential A...,Follow up with security and IT teams for deplo...
1,2025-02-06,"CTO from Manufacturing Software Provider, Pred...",- The company wants a vertical AI solution for...,Send follow-up email summarizing key discussio...


In [20]:
# ✅ **Step 10: Lead Scoring and Prioritization**
def score_lead(summary):
    """ Assign a lead score based on meeting summary content """
    if "high priority" in summary.lower() or "immediate interest" in summary.lower():
        return "Hot Lead (Score: 90+)"
    elif "follow-up needed" in summary.lower() or "budget discussion" in summary.lower():
        return "Warm Lead (Score: 70-89)"
    else:
        return "Cold Lead (Score: <70)"

df["Lead Score"] = df["Summary"].apply(score_lead)
print("✅ Lead Scoring Completed!")
df.head()

✅ Lead Scoring Completed!


,Date,Attendees,Summary,Next Steps,Lead Score
0,2025-02-05,"University IT Representative, Prediction Guard...",- The IT department is researching potential A...,Follow up with security and IT teams for deplo...,Cold Lead (Score: <70)
1,2025-02-06,"CTO from Manufacturing Software Provider, Pred...",- The company wants a vertical AI solution for...,Send follow-up email summarizing key discussio...,Cold Lead (Score: <70)
